In [10]:
# 随机态输入训练鉴别器
from mindquantum import *
from scipy.sparse import coo_matrix
import numpy as np
import mindspore as ms
from mindspore.nn import Adam, TrainOneStepCell
np.random.seed(1)

# 简单编码线路
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# # 振幅编码线路作为编码器
# base_encoder = Circuit()
# base_encoder.ry({"alpha_10":0.5}, 0).h(0).ry("alpha_20", 1).rz("lambda_20", 1).x(1).z(0,1)
# base_encoder.ry({"alpha_10":0.5}, 0).z(0,1).rx({"lambda_10":0.5}, 0).z(0,1).rx({"lambda_10":-0.5}, 0)
# base_encoder.z(0,1).x(1).ry({"alpha_11":-0.5}, 0).z(0,1).ry({"alpha_11":0.5}, 0).z(0,1)
# base_encoder.rx({"lambda_11":0.5}, 0).z(0,1).rx({"lambda_11":-0.5}, 0).z(0,1).h(0)
# encoder = base_encoder + shift(base_encoder, 2) # type:ignore
# encoder.as_encoder()

# 对鉴别器中 CZ 门增加的随机噪声
# disc_noise = np.random.normal(0, 0.02, 4)
disc_noise = np.array([-0.004981, 0.01088405, -0.00690972, 0.0080938]) # 一个噪声实例

# 鉴别器
disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz(disc_noise[0],0).rz(disc_noise[1], 2).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz(disc_noise[2],1).rz(disc_noise[3], 3).rz("theta2",1).rz("theta3",3).h(1).h(3)

circ = encoder + disc_circ

# 构建哈密顿量
measures = ['0111', '1011', '1101', '1110', '0101', '1010']
row = col = np.array([int(i, 2) for i in measures])
data = np.array([1.] * len(measures))
hams = Hamiltonian(coo_matrix((data, (row, col)), shape=(1 << 4, 1 << 4)).tocsr())

sim = Simulator("mqvector", 4)
grad_ops = sim.get_expectation_with_grad(hams, circ)
qlayer = MQLayer(grad_ops, weight="zeros")
opti = Adam(qlayer.trainable_params(), learning_rate=0.1)
train = TrainOneStepCell(qlayer, opti)

steps_num  = 501
batch_size = 4
alphas = np.random.rand(steps_num, batch_size, len(encoder.params_name)) * np.pi * 2

best_params = np.array([0.] * 4)
cost_his = []
min_cost = 1
for i in range(steps_num):
    data = ms.Tensor(alphas[i])
    res = float(train(data)[0,0]) # type:ignore
    cost_his.append(res)
    if res < min_cost:
        min_cost = res
        best_params = qlayer.weight.numpy()
    if i % 100 == 0:
        print(i, ': ', res)
        
print("best_params:\n", best_params)
print("min_cost:\n", min_cost)
print("cost_his:\n", cost_his)

0 :  1.5739729860797524e-05
100 :  4.953826646669768e-07
200 :  1.2169056873290174e-09
300 :  5.883902306352828e-13
400 :  1.2565686126914262e-15
500 :  1.1161522304405183e-18
best_params:
 [ 0.0049809995 -0.010884045   0.0069097285 -0.008093796 ]
min_cost:
 1.1161522304405183e-18
cost_his:
 [1.5739729860797524e-05, 0.0033140876330435276, 0.00012936998973600566, 0.000400000368244946, 0.0025302579160779715, 0.00147531321272254, 0.0001669037592364475, 8.279359462903813e-05, 4.192315827822313e-05, 0.0007192400516942143, 0.001261135214008391, 0.0004104413674212992, 0.0001046773322741501, 2.3840060748625547e-05, 0.00013855143333785236, 0.00012305221753194928, 0.0004117353819310665, 0.0003693024627864361, 0.000112027300929185, 2.9368358809733763e-05, 0.000361242622602731, 0.0004234064253978431, 0.0002415949129499495, 0.00031928298994898796, 4.396564690978266e-05, 5.051717016613111e-05, 8.278985478682443e-05, 0.00024270499125123024, 0.00012152452836744487, 9.513548866380006e-05, 5.06088072143

In [3]:
# 验证训练好的鉴别器的效果
from mindquantum import Circuit, Simulator, Hamiltonian, add_prefix, shift
from scipy.sparse import coo_matrix
import numpy as np

# 对鉴别器中 CZ 门增加的随机噪声
# noise = np.random.normal(0, 0.02, 4)
disc_noise = np.array([-0.004981, 0.01088405, -0.00690972, 0.0080938]) # 一个噪声实例
print('disc_noise:\n', disc_noise)

# 鉴别器
disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz(disc_noise[0],0).rz(disc_noise[1], 2).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz(disc_noise[2],1).rz(disc_noise[3], 3).rz("theta2",1).rz("theta3",3).h(1).h(3)

disc_weight = np.array([0.0049809995, -0.010884045,   0.0069097285, -0.008093796]) # 一个训练好的实例

disc_circ0 = disc_circ.apply_value(dict(zip(disc_circ.params_name, disc_weight))) # 训练好的鉴别器
disc_circ1 = disc_circ.apply_value(dict(zip(disc_circ.params_name, np.array([0.]*4)))) # 未训练的鉴别器

# 2-qubit amp_circuit 编译后的结果作为编码器
base_encoder = Circuit()
base_encoder.ry({"alpha_10":0.5}, 0).h(0).ry("alpha_20", 1).rz("lambda_20", 1).x(1).z(0,1)
base_encoder.ry({"alpha_10":0.5}, 0).z(0,1).rx({"lambda_10":0.5}, 0).z(0,1).rx({"lambda_10":-0.5}, 0)
base_encoder.z(0,1).x(1).ry({"alpha_11":-0.5}, 0).z(0,1).ry({"alpha_11":0.5}, 0).z(0,1)
base_encoder.rx({"lambda_11":0.5}, 0).z(0,1).rx({"lambda_11":-0.5}, 0).z(0,1).h(0)

encoder_0 = add_prefix(base_encoder, "0")
encoder_1_temp = add_prefix(base_encoder, "1")
encoder_1 = shift(encoder_1_temp, 2)

encoder = encoder_0 + encoder_1 # type:ignore
encoder.as_encoder()

circ0 = encoder + disc_circ0
circ1 = encoder + disc_circ1

# 哈密顿量
measures = ['0111', '1011', '1101', '1110', '0101', '1010']
row = col = np.array([int(i, 2) for i in measures])
data = np.array([1.] * len(measures))
hams = Hamiltonian(coo_matrix((data, (row, col)), shape=(1 << 4, 1 << 4)).tocsr())

# 随机量子态用于评估
sample_num = 100
alphas = np.random.rand(sample_num, len(encoder.params_name)) * np.pi * 2
sim = Simulator("mqvector", 4)

res = []
for alpha in alphas:
    pr=dict(zip(encoder.params_name, alpha))
    psi0 = encoder_0.get_qs(pr=pr)
    psi1 = encoder_1_temp.get_qs(pr=pr)
    fidelity = abs(sum(psi0.conj() * psi1)) ** 2 # type:ignore
    cost_0 = sim.get_expectation(hams, circ0, pr=pr).real
    cost_1 = sim.get_expectation(hams, circ1, pr=pr).real
    res.append((fidelity, cost_0, cost_1))

res = sorted(res, key=lambda x: x[0]) # 按照保真度进行排序

# 计算两个输入量子态保真度与测试失败概率之间的关系
fidelities = []
diffs0, diffs1 = [], []

for fid, cost_0, cost_1 in res:
    fidelities.append(fid)
    diffs0.append(abs(cost_0 - 0.5 + 0.5 * fid))
    diffs1.append(abs(cost_1 - 0.5 + 0.5 * fid))
    
print("fidelities:\n", fidelities)
print("diffs0:\n", diffs0)
print("diffs1:\n", diffs1)

np.set_printoptions(precision = 20)
print('mean0:', sum(diffs0) / len(res))
print('mean1:', sum(diffs1) / len(res))

disc_noise:
 [-0.004981    0.01088405 -0.00690972  0.0080938 ]
fidelities:
 [0.0016959819176776154, 0.0022884528446752432, 0.0031182912936407463, 0.00588545478354294, 0.012339689723122205, 0.013715345054973878, 0.014449599751256008, 0.0316066701382203, 0.03438304691911751, 0.03492872991134748, 0.038499882266480163, 0.0437928617661731, 0.0454685828296848, 0.05139506145308968, 0.05175688734370679, 0.05268289474580108, 0.057221575671389226, 0.0581129255202961, 0.05905067692324786, 0.06200876682070811, 0.06244505537666194, 0.06294641377559981, 0.07136882302575659, 0.07144532917397697, 0.07886843371607476, 0.08215453095927142, 0.08427980980426429, 0.08627697439771388, 0.09022437285929509, 0.09051710705463013, 0.09155458075949217, 0.09279465044774356, 0.09920001540338277, 0.0994456114094169, 0.103965289353476, 0.1057359880540633, 0.11045261176588744, 0.11762853380581824, 0.12158917158552215, 0.12891533061586233, 0.13060902560983162, 0.13610372234923307, 0.13645646287607782, 0.136475520165146

In [ ]:
# 非参数化量子线路等价性验证(一个固定门) 
# # 落后版本，不维护了

from mindquantum import *
from mindquantum.algorithm.library.amplitude_encoder import amp_circuit
from scipy.sparse import coo_matrix
import numpy as np
import mindspore as ms
from mindspore.nn import Adam, TrainOneStepCell
import matplotlib.pyplot as plt
np.random.seed(1)

# 简单编码线路
encoder = Circuit().h(0).h(1).h(2).h(3)
encoder.rx("alpha0", 0).rx("alpha1", 1).rx("alpha0", 2).rx("alpha1", 3)
encoder.as_encoder()

# # 2-qubit amp_circuit 编译后的结果作为编码器
# base_encoder = Circuit()
# base_encoder.ry({"alpha_10":0.5}, 0).h(0).ry("alpha_20", 1).rz("lambda_20", 1).x(1).z(0,1)
# base_encoder.ry({"alpha_10":0.5}, 0).z(0,1).rx({"lambda_10":0.5}, 0).z(0,1).rx({"lambda_10":-0.5}, 0)
# base_encoder.z(0,1).x(1).ry({"alpha_11":-0.5}, 0).z(0,1).ry({"alpha_11":0.5}, 0).z(0,1)
# base_encoder.rx({"lambda_11":0.5}, 0).z(0,1).rx({"lambda_11":-0.5}, 0).z(0,1).h(0)

# encoder_0 = add_prefix(base_encoder, "0")
# encoder_1_temp = add_prefix(base_encoder, "1")
# encoder_1 = shift(encoder_1_temp, 2)

# encoder = encoder_0 + encoder_1 # type:ignore
# encoder.as_encoder()

# 参考线路的随机噪声
# core_noise = np.random.normal(0, 0.02, 8) # 有噪声
core_noise = np.array([0.05472199, 0.00614403, 0.02152583, -0.01514881, # 一个噪声实例
                       0.02607528, 0.02897665, -0.00584025, -0.0216056])
# core_noise = np.array([0.] * 8) # 无噪声
print("core_noise:", core_noise)

# 参考线路
ref_circ = Circuit()
ref_circ.x(1).z(1, 0).rz(core_noise[0], 0).rz(core_noise[1], 1).x(1)
ref_circ.x(0).z(1, 0).rz(core_noise[2], 0).rz(core_noise[3], 1).x(0)
ref_circ.z(1, 0).rz(core_noise[4], 0).rz(core_noise[5], 1)

# 生成器线路
gen_circ0 = Circuit().x(2).x(3).z(3,2).rz(core_noise[6], 2).rz(core_noise[7], 3).x(2).x(3)
gen_circ1 = gen_circ0 + H.on(2)

core_circ0 = ref_circ + gen_circ0 # 相同的线路
core_circ1 = ref_circ + gen_circ1 # 不同的线路

disc_noise = np.array([-0.004981, 0.01088405, -0.00690972, 0.0080938]) # 一个噪声实例
disc_weight = np.array([ 0.004981,  -0.01088404,  0.00691002, -0.00809349]) # 一个训练好的实例

disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz(disc_noise[0],0).rz(disc_noise[1], 2).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz(disc_noise[2],1).rz(disc_noise[3], 3).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, disc_weight))) # 训练好的鉴别器

circ0 = encoder + core_circ0 + disc_circ # type_ignore
circ1 = encoder + core_circ1 + disc_circ # type_ignore

# 哈密顿量
measures = ['0111', '1011', '1101', '1110', '0101', '1010']
row = col = np.array([int(i, 2) for i in measures])
data = np.array([1.] * len(measures))
hams = Hamiltonian(coo_matrix((data, (row, col)), shape=(1 << 4, 1 << 4)).tocsr())

# 随机量子态用于评估
sample_num = 100
params = np.random.rand(sample_num, len(encoder.params_name)) * np.pi * 2
sim = Simulator("mqvector", 4)

probs0 = []
probs1 = []
for param in params:
    pr = dict(zip(circ0.params_name, param))
    prob0 = sim.get_expectation(hams, circ0, pr=pr).real
    prob1 = sim.get_expectation(hams, circ1, pr=pr).real
    probs0.append(prob0)
    probs1.append(prob1)

mean0 = sum(probs0) / len(probs0)
mean1 = sum(probs1) / len(probs1)
print("mean0:\n", mean0)
print("mean1:\n", mean1)

print("fidelity0:\n", 1 - 2 * mean0)
print("fidelity1:\n", 1 - 2 * mean1)

core_noise: [ 0.05472199  0.00614403  0.02152583 -0.01514881  0.02607528  0.02897665
 -0.00584025 -0.0216056 ]
mean0:
 0.0003809829071418183
mean1:
 0.49971660095087045
fidelity0:
 0.9992380341857163
fidelity1:
 0.0005667980982591025


In [14]:
# 非参数化量子线路等价性验证（一个参数门）

from mindquantum import Circuit, Simulator, Hamiltonian
from scipy.sparse import coo_matrix
import numpy as np
np.random.seed(1)

# 简单编码线路
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# # 2-qubit amp_circuit 编译后的结果作为编码器
# base_encoder = Circuit()
# base_encoder.ry({"alpha_10":0.5}, 0).h(0).ry("alpha_20", 1).rz("lambda_20", 1).x(1).z(0,1)
# base_encoder.ry({"alpha_10":0.5}, 0).z(0,1).rx({"lambda_10":0.5}, 0).z(0,1).rx({"lambda_10":-0.5}, 0)
# base_encoder.z(0,1).x(1).ry({"alpha_11":-0.5}, 0).z(0,1).ry({"alpha_11":0.5}, 0).z(0,1)
# base_encoder.rx({"lambda_11":0.5}, 0).z(0,1).rx({"lambda_11":-0.5}, 0).z(0,1).h(0)

# encoder_0 = add_prefix(base_encoder, "0")
# encoder_1_temp = add_prefix(base_encoder, "1")
# encoder_1 = shift(encoder_1_temp, 2)

# encoder = encoder_0 + encoder_1 # type:ignore
# encoder.as_encoder()

# 参考线路的随机噪声
# core_noise = np.random.normal(0, 0.02, 8) # 有噪声
core_noise = np.array([0.05472199, 0.00614403, 0.02152583, -0.01514881, # 一个噪声实例
                       0.02607528, 0.02897665, -0.00584025, -0.0216056])
core_noise = np.array([0.] * 8) # 无噪声
print("core_noise:", core_noise)

# 参考线路
ref_circ = Circuit()
ref_circ.x(1).z(1, 0).rz(core_noise[0], 0).rz(core_noise[1], 1).x(1)
ref_circ.x(0).z(1, 0).rz(core_noise[2], 0).rz(core_noise[3], 1).x(0)
ref_circ.z(1, 0).rz(core_noise[4], 0).rz(core_noise[5], 1)

# 生成器线路
gen_circ = Circuit().x(2).x(3).z(3,2).rz(core_noise[6], 2).rz(core_noise[7], 3).x(2).x(3).ry("delta", 2)

disc_noise = np.array([-0.004981, 0.01088405, -0.00690972, 0.0080938]) # 一个噪声实例
weight = np.array([ 0.004981,  -0.01088404,  0.00691002, -0.00809349]) # 一个训练好的实例

disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz(disc_noise[0],0).rz(disc_noise[1], 2).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz(disc_noise[2],1).rz(disc_noise[3], 3).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, weight))) # 训练好的鉴别器

circ = encoder + ref_circ + gen_circ + disc_circ # type_ignore

# 哈密顿量
measures = ['0111', '1011', '1101', '1110', '0101', '1010']
row = col = np.array([int(i, 2) for i in measures])
data = np.array([1.] * len(measures))
hams = Hamiltonian(coo_matrix((data, (row, col)), shape=(1 << 4, 1 << 4)).tocsr())

# 随机量子态用于评估
sample_num = 100
alphas = np.random.rand(sample_num, len(encoder.params_name)) * np.pi * 2
sim = Simulator("mqvector", 4)

costs_delta = []
deltas = []
for i in range(101):
    delta = i * 2 * np.pi / 100
    deltas.append(delta)
    costs_alphas = []
    for alpha in alphas:
        pr = dict(zip(encoder.params_name, alpha))
        pr.update({"delta":delta})
        costs_alphas.append(sim.get_expectation(hams, circ, pr=pr).real)
    costs_delta.append(sum(costs_alphas) / len(costs_alphas))

print("deltas:\n", deltas)
print("costs_delta:\n", costs_delta)

core_noise: [0. 0. 0. 0. 0. 0. 0. 0.]
deltas:
 [0.0, 0.06283185307179587, 0.12566370614359174, 0.18849555921538758, 0.25132741228718347, 0.3141592653589793, 0.37699111843077515, 0.43982297150257105, 0.5026548245743669, 0.5654866776461628, 0.6283185307179586, 0.6911503837897545, 0.7539822368615503, 0.8168140899333463, 0.8796459430051421, 0.9424777960769378, 1.0053096491487339, 1.0681415022205298, 1.1309733552923256, 1.1938052083641213, 1.2566370614359172, 1.3194689145077132, 1.382300767579509, 1.4451326206513047, 1.5079644737231006, 1.5707963267948966, 1.6336281798666925, 1.6964600329384882, 1.7592918860102842, 1.8221237390820801, 1.8849555921538756, 1.9477874452256716, 2.0106192982974678, 2.0734511513692637, 2.1362830044410597, 2.199114857512855, 2.261946710584651, 2.324778563656447, 2.3876104167282426, 2.4504422698000385, 2.5132741228718345, 2.57610597594363, 2.6389378290154264, 2.701769682087222, 2.764601535159018, 2.827433388230814, 2.8902652413026093, 2.9530970943744057, 3.01592894

In [27]:
# 参考线路和生成线路只差一个整体相位 -1
enc = UN(H, 2)
ref_circ = enc + Circuit()
ref_circ.x(1).z(1, 0).x(1)
ref_circ.x(0).z(1, 0).x(0)
ref_circ.z(1, 0)

# 生成器线路
gen_circ0 = enc + Circuit().x(0).x(1).z(1, 0).x(0).x(1)

print(ref_circ.matrix())
print('lalal')
print(gen_circ0.matrix()*-1)
print(np.allclose(ref_circ.matrix(), gen_circ0.matrix()*-1))

[[ 0.5+0.j  0.5+0.j  0.5+0.j  0.5+0.j]
 [-0.5+0.j  0.5+0.j -0.5+0.j  0.5+0.j]
 [-0.5+0.j -0.5+0.j  0.5+0.j  0.5+0.j]
 [-0.5+0.j  0.5-0.j  0.5-0.j -0.5+0.j]]
lalal
[[ 0.5-0.j  0.5-0.j  0.5-0.j  0.5-0.j]
 [-0.5+0.j  0.5-0.j -0.5+0.j  0.5-0.j]
 [-0.5+0.j -0.5+0.j  0.5-0.j  0.5-0.j]
 [-0.5+0.j  0.5-0.j  0.5-0.j -0.5+0.j]]
True


In [16]:
# 参数化量子线路等价性验证

from mindquantum import *
from scipy.sparse import coo_matrix
import numpy as np
np.random.seed(1)

# 简单编码线路
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# 参考线路的随机噪声
# core_noise = np.random.normal(0, 0.02, 8) # 有噪声
# core_noise = np.array([0.05472199, 0.00614403, 0.02152583, -0.01514881, # 一个噪声实例
#                        0.02607528, 0.02897665, -0.00584025, -0.0216056])
core_noise = np.array([0.] * 8) # 无噪声
# print("core_noise:", core_noise)

beta = ParameterResolver("beta")
ref_circ = Circuit().ry(beta/2, 1).h(1).z(1, 0).rz(core_noise[0], 0).rz(core_noise[1], 1)
ref_circ.ry(beta/2,1).z(1,0).rz(core_noise[2], 0).rz(core_noise[3], 1).h(1)

gen_circ = Circuit().h(3).z(3,2).rz(core_noise[4], 2).rz(core_noise[5], 3)
gen_circ.ry(beta/2,3).z(3,2).rz(core_noise[6], 2).rz(core_noise[7], 3).h(3).ry(beta/2, 3).ry("delta",3)

disc_noise = np.array([-0.004981, 0.01088405, -0.00690972, 0.0080938]) # 一个噪声实例
weight = np.array([ 0.004981,  -0.01088404,  0.00691002, -0.00809349]) # 一个训练好的实例

disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz(disc_noise[0],0).rz(disc_noise[1], 2).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz(disc_noise[2],1).rz(disc_noise[3], 3).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, weight))) # 训练好的鉴别器

circ = encoder + ref_circ + gen_circ + disc_circ
circ.as_encoder()

# 哈密顿量
measures = ['0111', '1011', '1101', '1110', '0101', '1010']
row = col = np.array([int(i, 2) for i in measures])
data = np.array([1.] * len(measures))
hams = Hamiltonian(coo_matrix((data, (row, col)), shape=(1 << 4, 1 << 4)).tocsr())

# 编码器随机参数
sample_num_alpha = 10
alphas = np.random.rand(sample_num_alpha, len(encoder.params_name)) * np.pi * 2

# 参数化线路随机参数
sample_num_beta = 10
betas = np.random.rand(sample_num_beta) * np.pi * 2
sim = Simulator("mqvector", 4)

costs_delta = []
deltas = []
for i in range(101):
    delta = i * 2 * np.pi / 100
    deltas.append(delta)
    pr = {"delta":delta}
    costs_alphas = []
    for alpha in alphas:
        costs_beta = []
        pr.update(dict(zip(encoder.params_name, alpha)))
        for beta in betas:
            pr.update({"beta":beta})
            costs_beta.append(sim.get_expectation(hams, circ, pr=pr).real)
        costs_alphas.append(sum(costs_beta) / len(costs_beta))
    costs_delta.append(sum(costs_alphas) / len(costs_alphas))

print("deltas:\n", deltas)
print("costs_delta:\n", costs_delta)

deltas:
 [0.0, 0.06283185307179587, 0.12566370614359174, 0.18849555921538758, 0.25132741228718347, 0.3141592653589793, 0.37699111843077515, 0.43982297150257105, 0.5026548245743669, 0.5654866776461628, 0.6283185307179586, 0.6911503837897545, 0.7539822368615503, 0.8168140899333463, 0.8796459430051421, 0.9424777960769378, 1.0053096491487339, 1.0681415022205298, 1.1309733552923256, 1.1938052083641213, 1.2566370614359172, 1.3194689145077132, 1.382300767579509, 1.4451326206513047, 1.5079644737231006, 1.5707963267948966, 1.6336281798666925, 1.6964600329384882, 1.7592918860102842, 1.8221237390820801, 1.8849555921538756, 1.9477874452256716, 2.0106192982974678, 2.0734511513692637, 2.1362830044410597, 2.199114857512855, 2.261946710584651, 2.324778563656447, 2.3876104167282426, 2.4504422698000385, 2.5132741228718345, 2.57610597594363, 2.6389378290154264, 2.701769682087222, 2.764601535159018, 2.827433388230814, 2.8902652413026093, 2.9530970943744057, 3.015928947446201, 3.078760800517997, 3.14159265

In [4]:
# 变分量子线路优化：训练生成器
from mindquantum import *
from scipy.sparse import coo_matrix
import numpy as np
import mindspore as ms
from mindspore.nn import Adam, TrainOneStepCell
np.random.seed(1)

# 简单编码线路
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# 参考和生成线路的随机噪声
# core_noise = np.random.normal(0, 0.02, 8) # 有噪声
# core_noise = np.array([0.05472199, 0.00614403, 0.02152583, -0.01514881, # 一个噪声实例
#                        0.02607528, 0.02897665, -0.00584025, -0.0216056])
core_noise = np.array([0.] * 8) # 无噪声

# ref_circ = Circuit().h(0).z(1,0).rz(core_noise[0], 0).rz(core_noise[1], 1).h(0).h(1)
# ref_circ.z(1,0).rz(core_noise[2], 0).rz(core_noise[3], 1).h(1)

# gen_circ = Circuit().rz("beta0",2).ry("beta1",2).rz("beta2",2).rz("beta3",3).ry("beta4",3).rz("beta5",3)
# gen_circ.h(3).z(3,2).rz(core_noise[4], 2).rz(core_noise[5], 3).h(3)
# gen_circ.rz("beta6",2).ry("beta7",2).rz("beta8",2).rz("beta9",3).ry("beta10",3).rz("beta11",3)
# gen_circ.as_ansatz()

ref_circ = Circuit().x(0).x(1).z(1,0).rz(core_noise[0], 0).rz(core_noise[1], 1).x(0).x(1)
gen_circ = Circuit().rz("beta0",2).rz("beta1",3).z(3,2).rz(core_noise[2], 2).rz(core_noise[3], 3)
gen_circ.as_ansatz()

# 对鉴别器中 CZ 门增加的随机噪声
# disc_noise = np.random.normal(0, 0.02, 4)
disc_noise = np.array([-0.004981, 0.01088405, -0.00690972, 0.0080938]) # 一个噪声实例
disc_weight = np.array([ 0.004981,  -0.01088404,  0.00691002, -0.00809349]) # 一个训练好的实例

# 鉴别器
disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz(disc_noise[0],0).rz(disc_noise[1], 2).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz(disc_noise[2],1).rz(disc_noise[3], 3).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, disc_weight))) # 训练好的鉴别器

circ = encoder + ref_circ + gen_circ + disc_circ # type:ignore

# 构建哈密顿量
measures = ['0111', '1011', '1101', '1110', '0101', '1010']
row = col = np.array([int(i, 2) for i in measures])
data = np.array([1.] * len(measures))
hams = Hamiltonian(coo_matrix((data, (row, col)), shape=(1 << 4, 1 << 4)).tocsr())

sim = Simulator("mqvector", 4)
grad_ops = sim.get_expectation_with_grad(hams, circ)
qlayer = MQLayer(grad_ops, weight="zeros")
opti = Adam(qlayer.trainable_params(), learning_rate=0.1)
train = TrainOneStepCell(qlayer, opti)

steps_num  = 201
batch_size = 4
alphas = np.random.rand(steps_num, batch_size, len(encoder.params_name)) * np.pi * 2

best_params = np.array([0., 0.])
cost_his = []
min_cost = 1
for i in range(steps_num):
    res = train(ms.Tensor(alphas[i]))
    res = float(res.mean()) # type:ignore
    cost_his.append(res)
    if res < min_cost:
        min_cost = res
        best_params = qlayer.weight.numpy()
    if i % 100 == 0:
        print(i, ': ', res)

print("best_params:\n", best_params)
print("min_cost:\n", min_cost)
print("cost_his:\n", cost_his)

0 :  0.3327140212059021
100 :  3.6238841857993975e-05
200 :  1.7904947524627346e-09
best_params:
 [3.1414633 3.1416392]
min_cost:
 1.6374177569389303e-09
cost_his:
 [0.3327140212059021, 0.44540640711784363, 0.3568144738674164, 0.44205737113952637, 0.3994777500629425, 0.4878239333629608, 0.4413992762565613, 0.4689739942550659, 0.35316288471221924, 0.41294074058532715, 0.405827134847641, 0.315103679895401, 0.3889063596725464, 0.41158124804496765, 0.40214791893959045, 0.22857557237148285, 0.33634108304977417, 0.2740281820297241, 0.3116866946220398, 0.24100138247013092, 0.2263903021812439, 0.25103577971458435, 0.14731642603874207, 0.15038444101810455, 0.07729524374008179, 0.08348869532346725, 0.0715242400765419, 0.047400522977113724, 0.039360642433166504, 0.02179677225649357, 0.01142237801104784, 0.0035581679549068213, 0.00071001797914505, 4.178392555331811e-05, 0.0014338850742205977, 0.0027755380142480135, 0.008389461785554886, 0.00914772879332304, 0.016289260238409042, 0.0188258253037929

In [10]:
# 变分量子线路优化：检验训练好的生成器的效果
from mindquantum import *
from scipy.sparse import coo_matrix
import numpy as np
import mindspore as ms
np.random.seed(1)

# 简单编码线路
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

core_noise = np.array([0.] * 8) # 无噪声

ref_circ = Circuit().x(0).x(1).z(1,0).rz(core_noise[0], 0).rz(core_noise[1], 1).x(0).x(1)
gen_circ = Circuit().rz("beta0",2).rz("beta1",3).z(3,2).rz(core_noise[2], 2).rz(core_noise[3], 3)

# 对鉴别器中 CZ 门增加的随机噪声
# disc_noise = np.random.normal(0, 0.02, 4)
disc_noise = np.array([-0.004981, 0.01088405, -0.00690972, 0.0080938]) # 一个噪声实例
disc_weight = np.array([0.004981, -0.01088404, 0.00691002, -0.00809349]) # 一个训练好的实例

# 鉴别器
disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz(disc_noise[0],0).rz(disc_noise[1], 2).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz(disc_noise[2],1).rz(disc_noise[3], 3).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, disc_weight))) # 训练好的鉴别器

circ = encoder + ref_circ + gen_circ + disc_circ # type:ignore

# 构建哈密顿量
measures = ['0111', '1011', '1101', '1110', '0101', '1010']
row = col = np.array([int(i, 2) for i in measures])
data = np.array([1.] * len(measures))
hams = Hamiltonian(coo_matrix((data, (row, col)), shape=(1 << 4, 1 << 4)).tocsr())

sim = Simulator("mqvector", 4)

sample_num  = 100
alphas = np.random.rand(steps_num,  len(encoder.params_name)) * np.pi * 2

gen_weight = np.array([0.0, 0.0]) # 未训练
costs = []
circ_0 = circ.apply_value(dict(zip(gen_circ.params_name, gen_weight)))
for i in range(sample_num):
    pr = dict(zip(encoder.params_name, alphas[i]))
    cost = sim.get_expectation(hams, circ_0, pr=pr).real
    costs.append(cost)

print("cost:\n", np.array(costs).mean())

gen_weight = np.array([3.1414633, 3.1416392]) # 训练好的一个实例
costs = []
circ_1 = circ.apply_value(dict(zip(gen_circ.params_name, gen_weight)))
for i in range(sample_num):
    pr = dict(zip(encoder.params_name, alphas[i]))
    cost = sim.get_expectation(hams, circ_1, pr=pr).real
    costs.append(cost)

print("cost:\n", np.array(costs).mean())


cost:
 0.4230747982856608
cost:
 1.3995773554979896e-09
